In [0]:
%sql
use catalog emrcatalog;

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# read data from bronze layer
df_hosa=spark.read.parquet('/mnt/bronze/hosa/departments')
df_hosb=spark.read.parquet('/mnt/bronze/hosb/departments')


In [0]:
df_merged=df_hosa.unionByName(df_hosb)


In [0]:
df_merged=df_merged.withColumn('Source_Dept_ID',col('DeptID'))\
    .withColumn('Dept_ID',concat(col('DeptID'),lit('-'),col('datasource')))\
    .drop('DeptID')


In [0]:
display(df_merged.limit(2))

In [0]:
df_merged.createOrReplaceGlobalTempView('departments')

In [0]:
%sql
show views in global_temp;

In [0]:
%sql
create schema if not exists emrcatalog.silver;

In [0]:
%sql
create table if not exists emrcatalog.silver.departments 
(
  Dept_ID string,
  Source_Dept_ID string,
  Name string,
  datasource string,
  is_quarantine boolean -- if some rows fail data quailty check, we quarantine them, keep them seperate
)
using delta;

In [0]:
%sql
truncate table emrcatalog.silver.departments

In [0]:
%sql
insert into emrcatalog.silver.departments
select Dept_ID,Source_Dept_ID,Name,datasource,
case when Source_Dept_ID is NULL or Name is NULL then true else false end as is_quarantine
from global_temp.departments;

In [0]:
%sql
select is_quarantine,count(*) from emrcatalog.silver.departments group by is_quarantine